In [1]:
import sys
import numpy as np
import cv2

In [ ]:
# readNet(model[, config[, framework]]) -> retval
net = cv2.dnn.readNet()

# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
blob = cv2.dnn.blobFromImage

In [7]:
import os
file_lists = os.listdir('./googlenet_old/fig/')
print(file_lists)

file_names = []
for i in file_lists:
    file_name = './googlenet_old/fig/' + i
    file_names.append(file_name)

#filename = './googlenet_old/fig/pict/restmb_idxmake.jfif'

img = cv2.imread(file_name)

if img is None:
    print('Image load failed!')
    sys.exit()

    
## dnn 모델 #####################################################
model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet_old/deploy.prototxt'
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('net load failed!')
    sys.exit()
    
    
# 클래스 이름 불러오기

classNames = []
with open('./googlenet_old/classification_classes_ILSVRC2012.txt','rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

idx = 0

while True:
    print(classNames[0])    

    # blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
    blob = cv2.dnn.blobFromImage(img, 1, (224,224),(104,117,123),
                                swapRB = False)
    net.setInput(blob)
    prob = net.forward()
    ##################################################################

    print(prob.shape)
    print(type(prob))

    out = prob.flatten()
    print(out.shape)
    classid = np.argmax(out)
    print(classid)
    confidence = out[classid]
    print(confidence)
    category = classNames[classid]
    print(category)

    text = f'{category}  정확도 : {confidence*100:4.2f}%'
    cv2.putText(img, text, (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.8,
               (0,0,255),2,cv2.LINE_AA)

    cv2.imshow('blob', blob)
    if cv2.waitKey(2000) == 27:
        break
        
    idx += 1
    if idx >= len(file_names):
        idx = 0
    
cv2.destroyAllWindows()

['apple1.png', 'apple2.png', 'beagle.jpg', 'fish.jpg', 'pict', 'pineapple.jpg', 'scooter.jpg', 'space_shuttle.jpg']
tench, Tinca tinca
(1, 1000)
<class 'numpy.ndarray'>
(1000,)
812
0.99993455
space shuttle


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:124: error: (-215:Assertion failed) bmi && width >= 0 && height >= 0 && (bpp == 8 || bpp == 24 || bpp == 32) in function 'FillBitmapInfo'


In [2]:
## 연습 구조 에러

import sys
import numpy as np
import cv2

img = cv2.imread('./opencv_face_detector/fig/sunglass.png')

if img is None:
    print('image read failed')
    sys.exit()
    
model = './opencv_face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = './opencv_face_detector/deploy.prototxt'

face_net = cv2.dnn.readNet(model, config)

if face_net.empty():
    print('Net open failed')
    sys.exit()

blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                            swapRB=False)

face_net.setInput(blob)
out = face_net.forward()
print(out.shape)

detect = out[0,0,:,:]
print(detect.shape)


h, w = img.shape[:2]

threshold = 0.5
for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > threshold:
        x1 = (detect[i, 3]*w)
        y1 = (detect[i, 4]*h)
        x2 = (detect[i, 5]*w)
        y2 = (detect[i, 6]*h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2),
                     (0, 0, 255))
        
        text = 'Face: {}%'.format(round(confidence, 2))
        cv2.putText(img, text, (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX,
                   0.8, (0, 0, 255), 1, cv2.LINE_AA)


cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

(1, 1, 200, 7)
(200, 7)


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


In [4]:
import numpy as np
import sys
import cv2

img = cv2.imread('./opencv_face_detector_old/fig/sunglass.png')

## Tensorflow
model = './opencv_face_detector/opencv_face_detector_uint8.pb'
config = './opencv_face_detector/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Video open failed')
    cap.release()
    sys.exit()

face_net = cv2.dnn.readNet(model, config)

if face_net.empty():
    print('net load failed')
    sys.exit()
    
while True:
    ret, img = cap.read()
    if not ret:
        print('Video frame failed')
        break
        
    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                                swapRB = False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0, 0, :, :]
    # print(detect.shape)

    h, w = img.shape[:2]
    threshold = 0.5
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]

        if confidence > threshold:
            x1 = int(detect[i,3]*w)
            y1 = int(detect[i,4]*h)
            x2 = int(detect[i,5]*w)
            y2 = int(detect[i,6]*h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    #         text = f'Face: {confidence*100:4.2f}%'
            text = 'Face: {}%'.format(round(confidence*100, 2))
            cv2.putText(img, text, (x1, y1-3), cv2.FONT_HERSHEY_COMPLEX,
                       1, (255, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('image', img)
    
    if cv2.waitKey(20) ==  27:
        break

cap.release()
cv2.destroyAllWindows()